In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 14.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747923 sha256=746dc4949ad7f88c924365c85033436116a082f6f6911711fd65d8527217e334
  Stored in directory: /root/.cache/pip/wheels/0c/7f/b4/0e68c6d8d89d2e582e5498ad88616c16d7c19028680e9d3840
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, count, min, max

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder.appName("MovieLensRecommendation").getOrCreate()

In [ ]:
# Load MovieLens Dataset (33M ratings)
ratings_path = "/content/ratings.csv"
movies_path = "/content/movies.csv"
ratings = spark.read.csv(ratings_path, header=True, inferSchema=True)
movies = spark.read.csv(movies_path, header=True, inferSchema=True)

In [ ]:
# Convert timestamps to human-readable format
ratings = ratings.withColumn("date", from_unixtime(col("timestamp")).cast("timestamp")).drop("timestamp")

In [ ]:
# Handle missing values
ratings = ratings.dropna()

In [ ]:
# Filter out cold-start users and movies
movies_with_enough_ratings = ratings.groupBy("movieId").agg(count("rating").alias("num_ratings"))
ratings = ratings.join(movies_with_enough_ratings, "movieId").filter(col("num_ratings") >= 10)
users_with_enough_ratings = ratings.groupBy("userId").agg(count("rating").alias("num_user_ratings"))
ratings = ratings.join(users_with_enough_ratings, "userId").filter(col("num_user_ratings") >= 10)
ratings = ratings.drop("num_ratings", "num_user_ratings")

In [ ]:
# Normalize ratings
min_rating = ratings.agg(min("rating")).collect()[0][0]
max_rating = ratings.agg(max("rating")).collect()[0][0]
ratings = ratings.withColumn("normalized_rating", (col("rating") - min_rating) / (max_rating - min_rating))

# Save cleaned dataset as PySpark DataFrame for further model training
ratings.show(5)

+------+-------+------+-------------------+------------------+
|userId|movieId|rating|               date| normalized_rating|
+------+-------+------+-------------------+------------------+
|     1|      1|   4.0|2008-11-03 17:52:19|0.7777777777777778|
|     1|    110|   4.0|2008-11-05 06:04:46|0.7777777777777778|
|     1|    158|   4.0|2008-11-03 17:31:43|0.7777777777777778|
|     1|    260|   4.5|2008-11-03 18:00:04|0.8888888888888888|
|     1|    356|   5.0|2008-11-03 17:58:39|               1.0|
+------+-------+------+-------------------+------------------+
only showing top 5 rows



In [ ]:
import time
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Start time measurement
start_time = time.time()

# Train ALS Model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
als_model = als.fit(ratings)

# End time measurement
end_time = time.time()

# Print execution time
print(f"ALS Model Training Time: {end_time - start_time:.4f} seconds")

ALS Model Training Time: 32.6606 seconds


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

# Generate predictions
predictions = als_model.transform(ratings)

# Define evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse_als = evaluator.evaluate(predictions)

evaluator = RegressionEvaluator(metricName="mse", labelCol="rating", predictionCol="prediction")
mse_als = evaluator.evaluate(predictions)

print(f"ALS Model -> RMSE: {rmse_als}, MSE: {mse_als}")

ALS Model -> RMSE: 0.7303235042669299, MSE: 0.5333724208847284


In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00


In [ ]:
# Train Deep Learning Models (NCF, Autoencoders)
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense

In [ ]:
import time
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense

# Start time measurement
start_time = time.time()

# Define input layers
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Embedding layers for users and movies
user_embedding = Embedding(input_dim=num_users, output_dim=50)(user_input)
movie_embedding = Embedding(input_dim=num_movies, output_dim=50)(movie_input)

# Flatten the embeddings
user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)

# Compute dot product
dot_product = Dot(axes=1)([user_vec, movie_vec])

# Output layer
output = Dense(1, activation='linear')(dot_product)

# Build Model
ncf_model = Model([user_input, movie_input], output)
ncf_model.compile(optimizer='adam', loss='mse')

# Generate training data
train_users = np.random.randint(0, num_users, size=(100000,))
train_movies = np.random.randint(0, num_movies, size=(100000,))
train_ratings = np.random.rand(100000)

# Train Model
ncf_model.fit([train_users, train_movies], train_ratings, epochs=10, batch_size=64)

# End time measurement
end_time = time.time()

# Print execution time
print(f"NCF Model Training Time: {end_time - start_time:.4f} seconds")

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 0.1710
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0641
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0115
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0036
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.0041
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.0067
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.0057
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0038
Epoch 9/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0037
Epoch 10/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.0041
NCF Model Training Time: 105.1360 seconds


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Generate predictions using the trained NCF model
predicted_ratings = ncf_model.predict([train_users, train_movies])

# Compute RMSE and MSE
mse_ncf = mean_squared_error(train_ratings, predicted_ratings)
rmse_ncf = np.sqrt(mse_ncf)

print(f"NCF Model -> RMSE: {rmse_ncf}, MSE: {mse_ncf}")


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
NCF Model -> RMSE: 0.06327373540707314, MSE: 0.0040035655923643014
